## Welcome

This is material for the **Panel Data** chapter in Scott Cunningham's book, [Causal Inference: The Mixtape.](https://mixtape.scunning.com/)


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels import PanelOLS
import plotnine as p

In [2]:
# read data
def read_data(file):
    return pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/" + file)

In [3]:
sasp = read_data("sasp_panel.dta")
#-- Delete all NA
sasp = sasp.dropna()

#-- order by id and session 
sasp.sort_values('id', inplace=True)


In [4]:
#Balance Data
times = len(sasp.session.unique())
in_all_times = sasp.groupby('id')['session'].apply(lambda x : len(x)==times).reset_index()
in_all_times.rename(columns={'session':'in_all_times'}, inplace=True)
balanced_sasp = pd.merge(in_all_times, sasp, how='left', on='id')
balanced_sasp = balanced_sasp[balanced_sasp.in_all_times]
balanced_sasp.shape

(1028, 32)

In [5]:
provider_second = np.zeros(balanced_sasp.shape[0])
provider_second[balanced_sasp.provider_second == "2. Yes"] = 1
balanced_sasp.provider_second = provider_second

In [6]:
#Demean Data
features = balanced_sasp.columns.to_list()
features = [x for x in features if x not in ['session', 'id', 'in_all_times']]
demean_features = ["demean_{}".format(x) for x in features]
balanced_sasp[demean_features] = balanced_sasp.groupby('id')[features].apply(lambda x : x - np.mean(x))

In [7]:
balanced_sasp

,id,in_all_times,session,age,age_cl,appearance_cl,bmi,schooling,asq_cl,provider_second,...,demean_hispanic,demean_other,demean_white,demean_asq,demean_cohab,demean_married,demean_divorced,demean_separated,demean_nevermarried,demean_widowed
3,6.0,True,3.0,29.0,45.0,4.0,30.893555,16.0,2025.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.0,True,1.0,29.0,32.5,6.0,30.893555,16.0,1056.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6.0,True,2.0,29.0,30.0,8.0,30.893555,16.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6.0,True,4.0,29.0,21.0,6.0,30.893555,16.0,441.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,8.0,True,3.0,25.0,37.0,5.0,22.886999,14.0,1369.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,684.0,True,2.0,28.0,30.0,6.0,27.435720,12.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1495,690.0,True,3.0,37.0,35.0,5.0,19.366392,14.0,1225.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,690.0,True,2.0,37.0,30.0,6.0,19.366392,14.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,690.0,True,4.0,37.0,45.0,8.0,19.366392,14.0,2025.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Pooled OLS

In [8]:
dep_var = "+".join(features)
formula = """lnw ~ age + asq + bmi + hispanic + black + other + asian + schooling + cohab + 
            married + divorced + separated + age_cl + unsafe + llength + reg + asq_cl + 
            appearance_cl + provider_second + asian_cl + black_cl + hispanic_cl + 
           othrace_cl + hot + massage_cl"""
ols = sm.OLS.from_formula(formula, data=balanced_sasp).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     17.39
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           3.97e-62
Time:                        13:32:50   Log-Likelihood:                -570.00
No. Observations:                1028   AIC:                             1192.
Df Residuals:                    1002   BIC:                             1320.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.0627      0.316     22.385      0.000       6.444       7.682
age                 0.0028      0.012      0.235      0.814      -0.020       0.026
asq                -0.0001      0.000     -0.828      0.408      -0.000       0.000
bmi                -0.0217      0.002     -9.296      0.000      -0.026      -0.017
hispanic           -0.2259      0.091     -2.472      0.014      -0.405      -0.047
black               0.0284      0.075      0.379      0.705      -0.119       0.175
other              -0.1116      0.061     -1.838      0.066      -0.231       0.008
asian               0.0862      0.154      0.559      0.576      -0.216       0.389
schooling           0.0198      0.010      1.997      0.046       0.000       0.039
cohab              -0.0540      0.040     -1.347      0.178      -0.133       0.025
married             0.0046      0.042      0.110      0.912      -0.078       0.087
divorced           -0.0209      0.039     -0.539      0.590      -0.097       0.055
separated          -0.0557      0.063     -0.881      0.378      -0.180       0.068
age_cl             -0.0013      0.009     -0.154      0.877      -0.018       0.016
unsafe              0.0134      0.028      0.473      0.636      -0.042       0.069
llength            -0.3083      0.020    -15.517      0.000      -0.347      -0.269
reg                -0.0470      0.028     -1.657      0.098      -0.103       0.009
asq_cl           4.401e-05   9.12e-05      0.482      0.630      -0.000       0.000
appearance_cl       0.0200      0.007      2.986      0.003       0.007       0.033
provider_second     0.0554      0.072      0.765      0.445      -0.087       0.197
asian_cl           -0.0135      0.059     -0.231      0.818      -0.129       0.102
black_cl            0.0919      0.063      1.465      0.143      -0.031       0.215
hispanic_cl         0.0517      0.076      0.677      0.499      -0.098       0.201
othrace_cl          0.1558      0.081      1.929      0.054      -0.003       0.314
hot                 0.1332      0.028      4.722      0.000       0.078       0.188
massage_cl         -0.1338      0.029     -4.585      0.000      -0.191      -0.077
==============================================================================
Omnibus:                       62.662   Durbin-Watson:                   1.095
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.620
Skew:                           0.425   Prob(JB):                     7.82e-26
Kurtosis:                       4.405   Cond. No.                     6.49e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.49e+04. This might indicate that the

#### Fixed Effects

In [9]:
balanced_sasp['y'] = balanced_sasp.lnw

In [10]:
balanced_sasp

,id,in_all_times,session,age,age_cl,appearance_cl,bmi,schooling,asq_cl,provider_second,...,demean_other,demean_white,demean_asq,demean_cohab,demean_married,demean_divorced,demean_separated,demean_nevermarried,demean_widowed,y
3,6.0,True,3.0,29.0,45.0,4.0,30.893555,16.0,2025.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.192957
4,6.0,True,1.0,29.0,32.5,6.0,30.893555,16.0,1056.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.585999
5,6.0,True,2.0,29.0,30.0,8.0,30.893555,16.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.585999
6,6.0,True,4.0,29.0,21.0,6.0,30.893555,16.0,441.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.010635
9,8.0,True,3.0,25.0,37.0,5.0,22.886999,14.0,1369.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.605170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,684.0,True,2.0,28.0,30.0,6.0,27.435720,12.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.396930
1495,690.0,True,3.0,37.0,35.0,5.0,19.366392,14.0,1225.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317
1496,690.0,True,2.0,37.0,30.0,6.0,19.366392,14.0,900.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317
1497,690.0,True,4.0,37.0,45.0,8.0,19.366392,14.0,2025.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.991465


In [11]:
formula = """lnw ~ -1 + C(id) + age + asq + bmi + hispanic + black + other + asian + schooling + 
                      cohab + married + divorced + separated + 
                      age_cl + unsafe + llength + reg + asq_cl + appearance_cl + 
                      provider_second + asian_cl + black_cl + hispanic_cl + 
                      othrace_cl + hot + massage_cl"""

ols = sm.OLS.from_formula(formula, data=balanced_sasp).fit(cov_type='cluster', 
                                                           cov_kwds={'groups': balanced_sasp['id']})
ols.summary()    

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 07 Mar 2021   Prob (F-statistic):                nan
Time:                        13:32:50   Log-Likelihood:                 162.25
No. Observations:                1028   AIC:                             215.5
Df Residuals:                     758   BIC:                             1548.
Df Model:                         269                                         
Covariance Type:              cluster                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
C(id)[6.0]         -0.4465      0.035    -12.683      0.000      -0.515      -0.377
C(id)[8.0]          0.3310      0.027     12.240      0.000       0.278       0.384
C(id)[10.0]         1.0513      0.040     26.539      0.000       0.974       1.129
C(id)[11.0]        -0.5627      0.026    -21.608      0.000      -0.614      -0.512
C(id)[18.0]         0.5518      0.034     16.025      0.000       0.484       0.619
C(id)[23.0]        -0.0312      0.038     -0.827      0.408      -0.105       0.043
C(id)[25.0]        -0.1525      0.035     -4.403      0.000      -0.220      -0.085
C(id)[29.0]         1.6517      0.068     24.285      0.000       1.518       1.785
C(id)[31.0]        -0.0586      0.020     -2.953      0.003      -0.098      -0.020
C(id)[33.0]         0.3145      0.021     14.914      0.000       0.273       0.356
C(id)[34.0]        -0.4864      0.030    -16.439      0.000      -0.544      -0.428
C(id)[35.0]         0.8514      0.040     21.275      0.000       0.773       0.930
C(id)[42.0]         0.8090      0.040     20.387      0.000       0.731       0.887
C(id)[44.0]        -0.3047      0.035     -8.649      0.000      -0.374      -0.236
C(id)[48.0]        -0.3631      0.030    -12.024      0.000      -0.422      -0.304
C(id)[53.0]         0.3511      0.031     11.184      0.000       0.290       0.413
C(id)[54.0]         0.6716      0.036     18.840      0.000       0.602       0.741
C(id)[56.0]        -0.4843      0.024    -20.443      0.000      -0.531      -0.438
C(id)[59.0]        -0.3581      0.025    -14.086      0.000      -0.408      -0.308
C(id)[61.0]        -0.4522      0.057     -7.929      0.000      -0.564      -0.340
C(id)[62.0]        -0.2078      0.033     -6.232      0.000      -0.273      -0.142
C(id)[64.0]        -0.3638      0.038     -9.589      0.000      -0.438      -0.289
C(id)[68.0]         0.2320      0.028      8.163      0.000       0.176       0.288
C(id)[72.0]         0.0063      0.033      0.191      0.849      -0.058       0.070
C(id)[73.0]        -0.1538      0.030     -5.065      0.000      -0.213      -0.094
C(id)[77.0]        -0.4928      0.037    -13.168      0.000      -0.566      -0.419
C(id)[78.0]        -0.5062      0.034    -14.938      0.000      -0.573      -0.440
C(id)[79.0]         0.9076      0.044     20.816      0.000       0.822       0.993
C(id)[83.0]        -0.4709      0.015    -30.601      0.000      -0.501      -0.441
C(id)[84.0]        -0.8525      0.037    -22.855      0.000      -0.926      -0.779
C(id)[86.0]         0.7815      0.036     21.836      0.000       0.711       0.852
C(id)[88.0]         0.5577      0.031     17.889      0.000       0.497       0.619
C(id)[91.0]        -0.7363      0.043    -16.985      0.000      -0.821      -0.651
C(id)[93.0]         0.2723      0.029      9.354      0.000

#### Demean OLS

In [12]:
#-- Demean OLS
dm_formula = """demean_lnw ~ demean_age + demean_asq + demean_bmi + 
                demean_hispanic + demean_black + demean_other +
                demean_asian + demean_schooling + demean_cohab + 
                demean_married + demean_divorced + demean_separated +
                demean_age_cl + demean_unsafe + demean_llength + demean_reg + 
                demean_asq_cl + demean_appearance_cl + 
                demean_provider_second + demean_asian_cl + demean_black_cl + 
                demean_hispanic_cl + demean_othrace_cl +
                demean_hot + demean_massage_cl"""

ols = sm.OLS.from_formula(dm_formula, data=balanced_sasp).fit(cov_type='cluster', 
                                                           cov_kwds={'groups': balanced_sasp['id']})
ols.summary()  


/Users/tcaputo/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 13


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             demean_lnw   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     32.39
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           9.91e-47
Time:                        13:32:50   Log-Likelihood:                 162.25
No. Observations:                1028   AIC:                            -296.5
Df Residuals:                    1014   BIC:                            -227.4
Df Model:                          13                                         
Covariance Type:              cluster                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept              -8.148e-09   1.44e-08     -0.565      0.572   -3.64e-08    2.01e-08
demean_age               4.48e-17   2.65e-18     16.927      0.000    3.96e-17       5e-17
demean_asq              1.008e-14   6.82e-16     14.766      0.000    8.74e-15    1.14e-14
demean_bmi              1.256e-17    7.3e-18      1.722      0.085   -1.74e-18    2.69e-17
demean_hispanic         1.353e-17   5.98e-18      2.265      0.024    1.82e-18    2.52e-17
demean_black            6.053e-18   6.48e-18      0.935      0.350   -6.64e-18    1.87e-17
demean_other            -3.41e-17   1.86e-17     -1.832      0.067   -7.06e-17    2.37e-18
demean_asian            2.047e-18   9.81e-18      0.209      0.835   -1.72e-17    2.13e-17
demean_schooling        1.101e-17   1.06e-17      1.042      0.298   -9.71e-18    3.17e-17
demean_cohab           -2.308e-17   1.16e-17     -1.982      0.047   -4.59e-17   -2.58e-19
demean_married          4.384e-17   1.25e-17      3.511      0.000    1.94e-17    6.83e-17
demean_divorced         2.291e-17   7.96e-18      2.878      0.004    7.31e-18    3.85e-17
demean_separated        9.066e-19   9.29e-18      0.098      0.922   -1.73e-17    1.91e-17
demean_age_cl              0.0023      0.007      0.338      0.735      -0.011       0.016
demean_unsafe              0.0510      0.028      1.794      0.073      -0.005       0.107
demean_llength            -0.4345      0.024    -17.758      0.000      -0.482      -0.387
demean_reg                -0.0373      0.019     -1.979      0.048      -0.074      -0.000
demean_asq_cl           -1.47e-05    7.6e-05     -0.194      0.847      -0.000       0.000
demean_appearance_cl       0.0056      0.006      0.965      0.334      -0.006       0.017
demean_provider_second     0.1131      0.061      1.862      0.063      -0.006       0.232
demean_asian_cl           -0.0099      0.034     -0.290      0.772      -0.077       0.057
demean_black_cl            0.0265      0.043      0.622      0.534      -0.057       0.110
demean_hispanic_cl        -0.0621      0.052     -1.193      0.233      -0.164       0.040
demean_othrace_cl          0.1422      0.050      2.867      0.004       0.045       0.239
demean_hot                 0.0525      0.027      1.924      0.054      -0.001       0.106
demean_massage_cl         -0.0010      0.028     -0.037      0.970      -0.056       0.054
==============================================================================
Omnibus:                       79.779   Durbin-Watson:                   2.528
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              361.895
Skew:                           0.172   Prob(JB):                     2.60e-79
Kurtosis:                       5.886   Cond. No.                     1.00e+16
=================================================

#### QUESTIONS
- Interpret the effect of natural log of session length on the natural log of hourly wage.  Describe the economic theory that might explain this relationship?  (HINT: Consider the role that supplier fixed versus variable costs may have on the hourly wage.)
- Becker described discrimination in terms of ``taste based``.  This meant that social interactions with people of the other race were factors into marginal cost.  Given that these persist, what does this imply about the effect that competition is having on discrimination?
- Hamermesh and Biddle suggest that beauty is valued on the market.  Describe some reasons why there is no effect on client beauty once we use the within estimators?
- What other interesting results did you find in this analysis?  Which ones surprised you and which ones were intuitive and why?